In [14]:
import re
import gensim
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models
from gensim.utils import simple_preprocess
from collections import Counter, defaultdict
from transformers import pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import os
os.chdir("../src/backend")
from question_generation.pipelines import pipeline as qg_pipeline

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../src/backend'

In [4]:
qg = qg_pipeline("question-generation", use_cuda=True)

In [5]:
def get_most_coherent_qa(qa):
    if len(qa) == 1:
        return qa[0]
    scores = [score(pair["question"] + " " + pair["answer"]) for pair in qa]
    idx = np.argmax(scores)
    return qa[idx]

In [6]:
import math
import torch
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel
# Load pre-trained model (weights)
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
def score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss=model(tensor_input, lm_labels=tensor_input)
    return math.exp(loss)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [7]:
os.chdir("../..")
with open("data/econ3_8.txt") as f:
    doc1 = f.read().strip()
#words = doc.split(' ')
#processed_doc = preprocess(doc)

In [8]:
with open("data/econ3_7.txt") as f:
    doc2 = f.read().strip()
#words = doc.split(' ')
#processed_doc = preprocess(doc)

In [9]:
# TODO filter meaningless sentences
def get_teacher_text(doc):
    "Extracts text spoken by teacher"
    matches = re.findall(r"(?:(?:[A-Z][a-z]*\s)*(?:[A-Z][a-z]*)):", doc)
    splits = re.split(r"(?:(?:[A-Z][a-z]*\s)*(?:[A-Z][a-z]*)):", doc)[1:]
    teacher = Counter(matches).most_common(1)[0][0]
    return ''.join([splits[i] for i in range(len(splits)) if matches[i] == teacher])

def clean(doc):
    return re.sub(r"[',]", "", re.sub(r"\s+", " ", doc))

def get_sents(doc):
    return [sent for sent in sent_tokenize(doc)]
def get_filtered_sents(doc):
    return [sent for sent in sent_tokenize(doc) if len(re.findall(r"\s", sent)) > 10]

In [10]:
sents = get_sents(clean(get_teacher_text(doc1)))
filtered_sents = get_filtered_sents(clean(get_teacher_text(doc1)))

In [39]:
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data_words = list(sent_to_words(filtered_sents))
# remove stop words
data_words = remove_stopwords(data_words)

id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

num_topics = 5

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
lda_model

2021-02-21 04:55:01.955 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2021-02-21 04:55:01.974 INFO    gensim.corpora.dictionary: built Dictionary(729 unique tokens: ['behind', 'feel', 'hiding', 'im', 'like']...) from 200 documents (total 2201 corpus positions)
2021-02-21 04:55:01.990 INFO    gensim.models.ldamodel: using symmetric alpha at 0.2
2021-02-21 04:55:01.991 INFO    gensim.models.ldamodel: using symmetric eta at 0.2
2021-02-21 04:55:01.992 INFO    gensim.models.ldamodel: using serial LDA version on this node
2021-02-21 04:55:01.997 INFO    gensim.models.ldamulticore: running online LDA training, 5 topics, 1 passes over the supplied corpus of 200 documents, updating every 6000 documents, evaluating every ~200 documents, iterating 50x with a convergence threshold of 0.001000
2021-02-21 04:55:01.997 WARNING gensim.models.ldamulticore: too few updates, training might not converge; consider increasing the number of passes or iterations to 

In [40]:
clusters = defaultdict(str)
labels = [np.array(lda_model.get_document_topics(sent))[:, 1].argmax() for sent in corpus]
confidences = [np.array(lda_model.get_document_topics(sent))[labels[i],1] for i, sent in enumerate(corpus)]
for i in range(len(filtered_sents)):
    clusters[labels[i]] += filtered_sents[i] + " "
for topic in clusters:
    clusters[topic] = re.sub(r"\s+", " ", clusters[topic])
    clusters[topic] = re.sub(r"[',]", "", clusters[topic])
clusters

defaultdict(str,
            {2: 'Looks like I feel Im only hiding my tiredness behind my makeup. So two students who were in a dorm on campus had to evacuate because of a fire alarm. Ill situate one during an online exam because theres so much drama. I think pertaining to this but hopefully you figured out you didnt need to know that for the midterm. So these are just some useful tools that will start with and then well start the formal chapter 19 right after this. So in this case we would take 200 times 1.04 squared to get that same number right now that second way of doing it is useful because we dont have to go step by step through every year. In fact We can say in general if we want to know what output is in your team. Okay so its a really useful formula for saying Okay if they have 4% growth and if they start out at this certain amount. Because a lot of times this is really useful to be able to answer this question. Lets do the next algebraic step were going to take one plus a To

In [41]:
df = pd.DataFrame().assign(topic=labels, confidence=confidences, sentences=filtered_sents)
top_sentences = df.groupby("topic").apply(lambda grp: df.loc[grp['confidence'].nlargest(1).index])
top_sentences = top_sentences.assign(questions=top_sentences["sentences"].apply(lambda sent: get_most_coherent_qa(qg(sent))))
top_sentences.index = top_sentences.index.droplevel(0)
top_sentences = top_sentences.reset_index(drop=True)
top_sentences.head()

2021-02-21 04:55:04.323 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


,topic,confidence,sentences,questions
0,0,0.979228,The 200 and we have 4% growth than by your one...,"{'answer': '4%', 'question': 'How much growth ..."
1,1,0.956942,Major government policy for this year includes...,{'answer': 'a decade long push to double incom...
2,2,0.955185,Now youve heard about the law of diminishing r...,"{'answer': 'the marginal product of labor', 'q..."
3,3,0.957396,This is in purchasing power parity 19 $90 Chin...,"{'answer': '1980', 'question': 'In what year w..."
4,4,0.959360,What does that mean okay it means We know that...,"{'answer': 'standards of living', 'question': ..."


In [ ]:
top_sentences

In [ ]:
questions = {}
for i in range(num_topics):
    questions[i] = {"questions":top_sentences[top_sentences["topic"] == i]["questions"].to_list()}
questions

In [20]:
from src.backend.nlp import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `init_models()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function init_models at 0x0000023AB8D2FA68>
```

Please see the `hash_funcs` [documentation]
(https://streamlit.io/docs/caching.html)
for more details.
     

In [21]:
summarizer = pipeline("summarization")

2021-02-21 04:21:23.275 INFO    transformers.configuration_utils: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/sshleifer/distilbart-cnn-12-6/config.json from cache at C:\Users\tanve/.cache\torch\transformers\90dbb6d4e8ba2227a79a0e5cc64eb216091a1d5c362a83c52c6c242a44d89b50.e1484cfc83ca98dbf8dbb660097e95a5ef503f348dd354064e58fa28f5e3ae16
2021-02-21 04:21:23.276 INFO    transformers.configuration_utils: Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim"

In [ ]:
summary_sentences = df.groupby("topic").apply(lambda grp: df.loc[grp['confidence'].nlargest(10).index])
summary_sentences.index = summary_sentences.index.droplevel(0)
summary_sentences = summary_sentences.reset_index(drop=True)
to_summarize = summary_sentences.groupby("topic")["sentences"].sum()

In [ ]:
summaries = to_summarize.apply(lambda x: ' '.join([t["summary_text"] for t in summarizer(x)])) 
summaries

In [ ]:
df.groupby("topic").count()

In [33]:
hash(qg)

-9223371883677288780

In [32]:
hash(summarizer)

-9223371883686807116

In [31]:
hash(tokenizer)

-9223371883709446924

In [30]:
hash(model)

-9223371883677284784

In [24]:
hash(stop_words)

TypeError: unhashable type: 'list'

In [37]:
id(None)

140708041546976